In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import jieba
import torch
import pickle
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from ark_nlp.nn import Ernie
from ark_nlp.nn import ErnieConfig
from ark_nlp.dataset import BIONERDataset
from ark_nlp.factory.task import BIONERTask
from ark_nlp.factory.optimizer import get_default_bert_optimizer
from ark_nlp.processor.tokenizer.transfomer import TokensTokenizer

In [ ]:
# 目录地址

train_data_path = '../data/source_datasets/CMeEE/CMeEE_train.json'
dev_data_path = '../data/source_datasets/CMeEE/CMeEE_dev.json'

### 一、数据读入与处理

#### 1. 数据读入

In [ ]:
train_data_df = pd.read_json(train_data_path)
dev_data_df = pd.read_json(dev_data_path)

In [ ]:
def gen_label(df):
    label = len(df['text']) * ['O']
    for entity_ in df['entities']:
        if entity_['end_idx'] > entity_['start_idx']:
            label[entity_['start_idx']] = 'B-'+ entity_['type']
            label[entity_['start_idx']+1: entity_['end_idx']+1] = ['I-'+ entity_['type']] * (entity_['end_idx'] - entity_['start_idx'])
        elif entity_['end_idx'] == entity_['start_idx']:
            label[entity_['start_idx']] = 'B-'+ entity_['type']
        else:
            continue
    return label

In [ ]:
train_data_df['label'] = train_data_df.apply(lambda x: gen_label(x), axis=1)
dev_data_df['label'] = dev_data_df.apply(lambda x: gen_label(x), axis=1)

In [ ]:
train_data_df = train_data_df.loc[:,['text', 'label']]
train_data_df['label'] = train_data_df['label'].apply(lambda x: str(x))
dev_data_df = dev_data_df.loc[:,['text', 'label']]
dev_data_df['label'] = dev_data_df['label'].apply(lambda x: str(x))

In [ ]:
ner_train_dataset = BIONERDataset(train_data_df)
ner_dev_dataset = BIONERDataset(dev_data_df)

#### 2. 词典创建和生成分词器

In [ ]:
# 可以先创建词典，再加载入分词器
# 也可以使用分词器自动加载
# bert_vocab = transformers.AutoTokenizer.from_pretrained('nghuyong/ernie-1.0')
# tokenizer = TransfomerTokenizer(bert_vocab, max_seq_len=30)

In [ ]:
tokenizer = TokensTokenizer(vocab='nghuyong/ernie-1.0', max_seq_len=100)

#### 3. ID化

In [ ]:
ner_train_dataset.convert_to_ids(tokenizer)
ner_dev_dataset.convert_to_ids(tokenizer)

<br>

### 二、模型构建

#### 1. 模型参数设置

In [ ]:
bert_config = ErnieConfig.from_pretrained('nghuyong/ernie-1.0', 
                                         num_labels=len(ner_train_dataset.cat2id))

#### 2. 模型创建

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dl_module = Ernie.from_pretrained('nghuyong/ernie-1.0/', 
                                  config=bert_config)

<br>

### 三、任务构建

#### 1. 任务参数和必要部件设定

In [ ]:
# 设置运行次数
num_epoches = 5
batch_size = 32

In [ ]:
optimizer = get_default_bert_optimizer(dl_module)

#### 2. 任务创建

In [ ]:
model = BIONERTask(dl_module, optimizer, 'ce', cuda_device=1)

#### 3. 训练

In [ ]:
model.fit(ner_train_dataset, 
          ner_dev_dataset,
          lr=3e-5,
          epochs=num_epoches, 
          batch_size=batch_size
         )

<br>

### 四、模型验证与保存

#### 1. 模型验证

In [ ]:
from ark_nlp.factory.predictor import BIONERPredictor

In [ ]:
ner_predictor_instance = BIONERPredictor(model.module, tokenizer, ner_train_dataset.cat2id)

In [ ]:
ner_predictor_instance.predict_one_sample('心悸伴电生理检查时可诱发持续性室上性心动过速者。')

#### 2. Batch模型验证

In [ ]:
test_data_df = pd.read_json('../data/source_datasets/CMeEE/CMeEE_test.json')
test_data_df = test_data_df.loc[:,['text']]

ner_test_dataset = BIONERDataset(test_data_df, categories=ner_train_dataset.categories, is_test=True)

In [ ]:
# predict_label = ner_predictor_instance.predict_batch(ner_test_dataset)

<br>

### 五、模型测试报告

In [ ]:
1. 基本功能测试 通过
2. one sample predict 通过
3. batch predict  无batch接口